The primary purpose of this notebook is to test the OpenEphys sorting "pipeline".
Specifically, after defining your pathnames appropriately, this notebook will use SpikeInterface to perform spike sorting, analyze the sorting quality, and exports the results in "Phy" format. This output is then passed into pyramid to extract behavioral data, eye data, and spiking data to create a "TrialFile" which can be easily converted in Matlab to a user-friendly format, like FIRA, to perform subsequent analyses.

In [15]:
import spikeinterface.widgets as sw
from AODR_session_sorters import OpenEphysSessionSorter as OES
from pyramid import cli
import pandas as pd
import os
%matplotlib ipympl

Below, we define the relevant paths. You should change these to match the corresponding paths on your machine. If you encounter issues while loading/sorting the file, chances are that your computer is having a hard time accessing the files from cloud storage. If that is the case, you need to transfer your files to a local directory, and also save the files to a local directory.

In [17]:
# Experiment directory top level - should contain the subfolders like ecodes, python, matlab.
expDir = "/Users/lowell/Documents/GitHub/Lab_Pipelines/experiments/aodr"
os.chdir(expDir) # regular python files will run from the first open folder in your tree, but notebooks will work from wherever the notebook is stored.
# Directory where the raw files are stored to be converted/sorted
dataSearchPath = "/Users/lowell/Library/CloudStorage/Box-Box/GoldLab/Data/Physiology/AODR/Data/Anubis/Raw/Behavior/"
# Where the rules for ecodes are stored
pyramidSearchPath = "/Users/lowell/Documents/GitHub/Lab_Pipelines/experiments/aodr/ecodes/"
# Conversion specifications
convertSpecs = "/Users/lowell/Documents/GitHub/Lab_Pipelines/experiments/aodr/AODR_experiment_LC.yaml"
# Base directory to save the output files from pyramid (hdf5 files)
baseSaveDir = "/Users/lowell/Library/CloudStorage/Box-Box/GoldLab/Data/Physiology/AODR/Data/Anubis/Converted/Behavior/Pyramid/"
# The Open Ephys session directory (technically not a file, but a folder)
currentFile = "Anubis_2024-06-25_13-40-57"
# Full directory to save the output files from pyramid (hdf5 files)
trialFileOutputName = baseSaveDir+currentFile+".hdf5"
# Directory to save the output files from sorting
sorted_out = dataSearchPath.split("Raw")[0]+"Sorted/"+currentFile 

Below, we run the sorter/analyzer. With ~4 channels of neural data and a 90 min recording, this should take between 10-20 minutes. The step_names variable defines the sequential processing steps that are defined as methods in AODR_session_sorters OpenEphysSessionSorter class. If you use the step "open_sigui", a gui window will open that allows to interact with the sorting results. Close the gui to continue to the next step and save the output.

In [6]:
sorter = OES(session_dir=dataSearchPath+currentFile, channel_names=[0,1,2,3],
             out_folder=sorted_out, 
             step_names=[
                 'read_data',
                 'set_tetrode',
                 'bandpass',
                 'run_one_sorter_and_analyzer',
                 'open_sigui',
                 'export_to_phy'
             ])

read_data ...OK
set_tetrode ...OK
bandpass ...OK
Geometry of the probe does not allow 1D drift correction


/Users/lowell/anaconda3/envs/gold_pipelines/lib/python3.11/site-packages/sklearn/covariance/_graph_lasso.py:191: ConvergenceWarning: graphical_lasso: did not converge after 100 iteration: dual gap: -6.046e-04
  warnings.warn(


write_memory_recording:   0%|          | 0/4429 [00:00<?, ?it/s]

detect peaks using locally_exclusive:   0%|          | 0/4429 [00:00<?, ?it/s]

extract waveforms shared_memory mono buffer:   0%|          | 0/4429 [00:00<?, ?it/s]

...

spike_amplitudes:   0%|          | 0/4429 [00:00<?, ?it/s]

run_one_sorter_and_analyzer ...OK
open_sigui ...OK


write_binary_recording:   0%|          | 0/4429 [00:00<?, ?it/s]

extract PCs:   0%|          | 0/4429 [00:00<?, ?it/s]

export_to_phy ...OK


Spike interface has a lot of widgets availble to use with python jupyter notebooks. These widgets are interactive and useful, and the below code will test whether they are working appropriately on your system.

In [7]:
w_ts_filt = sw.plot_traces(sorter.recording, channel_ids=[0,1,2,3], backend="ipywidgets")

AppLayout(children=(TimeSlider(children=(Dropdown(description='segment', options=(0,), value=0), Button(icon='…

Now, run Pyramid on the corresponding output to create a TrialFile. Make sure that the phy_reader.params_file points to the appropriate directory based on those you have defined above.

In [14]:
cli.main(["convert", 
        "--trial-file", trialFileOutputName, 
        "--search-path", pyramidSearchPath, 
        "--experiment", convertSpecs, 
        "--readers", 
        "ttl_reader.session_dir="+dataSearchPath+currentFile,
        "message_reader.session_dir="+dataSearchPath+currentFile,
        "gaze_x_reader.session_dir="+dataSearchPath+currentFile,
        "gaze_y_reader.session_dir="+dataSearchPath+currentFile,
        "pupil_reader.session_dir="+dataSearchPath+currentFile,
        "phy_reader.params_file="+sorted_out+"/phy/params.py"])

2024-08-02 13:26:21,881 [INFO] Pyramid 0.0.1
2024-08-02 13:26:21,892 [INFO] Using 6 readers.
2024-08-02 13:26:21,893 [INFO]   pyramid.neutral_zone.readers.open_ephys_session.OpenEphysSessionNumericEventReader
2024-08-02 13:26:21,896 [INFO]     Buffer ttl_1 using 1 transformers.
2024-08-02 13:26:21,897 [INFO]       pyramid.neutral_zone.transformers.standard_transformers.FilterRange
2024-08-02 13:26:21,897 [INFO]     Buffer ttl_2 using 1 transformers.
2024-08-02 13:26:21,898 [INFO]       pyramid.neutral_zone.transformers.standard_transformers.FilterRange
2024-08-02 13:26:21,899 [INFO]   pyramid.neutral_zone.readers.open_ephys_session.OpenEphysSessionTextEventReader
2024-08-02 13:26:21,902 [INFO]     Buffer messages using 1 transformers.
2024-08-02 13:26:21,902 [INFO]       udp_events.UDPEventParser
2024-08-02 13:26:21,903 [INFO]   pyramid.neutral_zone.readers.open_ephys_session.OpenEphysSessionSignalReader
2024-08-02 13:26:34,523 [INFO]     Buffer gaze_x using 1 transformers.
2024-08-02 

0